<a href="https://colab.research.google.com/github/hrishi483/ASR-using-LLMs-for-USR/blob/main/Fine_Tuning_using_custom_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio
!pip install accelerate -U
!pip install transformers[torch]

In [ ]:
!sudo apt-get install unzip


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unzip is already the newest version (6.0-26ubuntu3.1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [ ]:
!unzip -o -q '/content/English_data.zip'

[/content/English_data.zip]
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/English_data.zip or
        /content/English_data.zip.zip, and cannot find /content/English_data.zip.ZIP, period.


In [ ]:
!unzip /content/Hindi_Commonvoice_data-20230816T105523Z-001.zip

Archive:  /content/English_data.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/English_data.zip or
        /content/English_data.zip.zip, and cannot find /content/English_data.zip.ZIP, period.


In [ ]:
import glob
import re
path = glob.glob('/content/Hindi_Commonvoice_data/data/val/texts/*.txt')
audio_path = '/content/Hindi_Commonvoice_data/data/val/audios'

In [ ]:
path

[]

In [ ]:
data=[]
with open('val.csv', 'w') as g:
  g.write(f"text,path,audio\n")
  for paths in path:
    name = paths.split('/')[-1]
    name_audio = name.split('.')[0] + '.mp3'
    with open(paths, 'r') as f:
      for file in f:
        text = file.strip().lower()
        res = re.sub(r'[^\w\s]', '', text)
        g.write(f"{res},{audio_path},{audio_path}/{str(name_audio)}\n")

In [ ]:
!cat train.csv

cat: train.csv: No such file or directory


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from datasets import Dataset, Audio, Value, Features
from datasets import load_dataset

features = Features(
    {
        "text": Value("string"),
        'path': Value('string'),
        "audio": Audio(sampling_rate=16000)
    }
)
sample_data = load_dataset(
    'csv', data_files={
        'train': '/content/train.csv',
        'valid': '/content/val.csv'
    }
)


sample_data = sample_data.cast(features)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Casting the dataset:   0%|          | 0/1422 [00:00<?, ? examples/s]

In [ ]:
from transformers import WhisperFeatureExtractor
from transformers import WhisperTokenizer
from transformers import WhisperProcessor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-tiny")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-tiny", language="Assamese", task="transcribe")
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny", language="Assamese", task="transcribe")


Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
print(sample_data["train"][0])

FileNotFoundError: ignored

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["text"]).input_ids
    return batch

In [ ]:
custom_data = sample_data.map(prepare_dataset, remove_columns=sample_data.column_names["train"], num_proc=2)

Map (num_proc=2):   0%|          | 0/1422 [00:00<?, ? examples/s]

FileNotFoundError: ignored

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
import evaluate

metric = evaluate.load("wer")

In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")

In [ ]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="/content/whisper-tiny-as",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=100,
    max_steps=2000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=100,
    eval_steps=100,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
    use_cache = False
)

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=custom_data["train"],
    eval_dataset=custom_data["valid"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [ ]:
processor.save_pretrained(training_args.output_dir)

In [ ]:
trainer.train()

`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
100,0.574500,0.754312,70.253165
200,0.028300,0.941054,70.253165
300,0.004200,1.058047,67.721519
400,0.001300,1.174719,68.670886
500,0.000700,1.204368,68.987342
600,0.000400,1.232622,69.303797
700,0.000300,1.262360,68.987342
800,0.000200,1.285852,68.354430
900,0.000200,1.304402,68.987342
1000,0.000200,1.320763,69.620253
